In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
csv_path = "/content/drive/My Drive/CompeticionPLN/FinancES_train_kaggle.csv"

In [35]:
import pandas as pd
import numpy as np
import pandas as pd
import tensorflow as tf

# Cargar el CSV desde Google Drive
train_df = pd.read_csv(csv_path)
print(train_df.head())  # Ver las primeras filas

   id                                               text  label
0   0  Renfe afronta mañana un nuevo día de paros par...      2
1   1       Presupuesto populista con cimientos frágiles      2
2   2  Biden no cree que la OPEP+ vaya a ayudar con l...      2
3   3  La deuda de las familias cae en 25.000 millone...      0
4   4  Bestinver: no hay "momento más inoportuno" par...      2


In [36]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

# Convertir etiquetas en formato numérico si es necesario
train_df["label"] = train_df["label"].astype(int)

# Dividir en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(train_df["text"], train_df["label"], test_size=0.2, random_state=42)


In [37]:
sequence_lengths = train_df["text"].apply(lambda x: len(x.split()))
print(sequence_lengths.describe())  # Longitud promedio, mínima y máxima

count    6359.000000
mean       12.378833
std         3.725278
min         1.000000
25%        10.000000
50%        12.000000
75%        14.000000
max        35.000000
Name: text, dtype: float64


In [38]:
embeddings_path = "/content/drive/My Drive/CompeticionPLN/glove-sbwc.i25.vec"

from gensim.models import KeyedVectors

# Cargar embeddings de GloVe en un diccionario
def load_glove_embeddings(file_path):
    glove_embeddings = {}
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype="float32")
            glove_embeddings[word] = vector
    return glove_embeddings

glove_embeddings = load_glove_embeddings(embeddings_path)
print("Embeddings cargados:", len(glove_embeddings))


Embeddings cargados: 855381


In [39]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

# Parámetros
MAX_NUM_WORDS = 12000  # Máximo número de palabras en el vocabulario
EMBEDDING_DIM = 300   # Dimensión de los embeddings
MAX_SEQUENCE_LENGTH = 25  # Longitud máxima de las secuencias de entrada

# Tokenizar los textos
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_df["text"])  # Ajusta el tokenizador al corpus

# Obtener el índice de cada palabra
word_index = tokenizer.word_index

# Crear la matriz de embeddings
embedding_matrix = np.zeros((MAX_NUM_WORDS, EMBEDDING_DIM))

for word, i in word_index.items():
    if i < MAX_NUM_WORDS and word in glove_embeddings:
        embedding_matrix[i] = glove_embeddings[word]  # Asigna el vector de GloVe

print("Matriz de embeddings creada con tamaño:", embedding_matrix.shape)


Matriz de embeddings creada con tamaño: (12000, 300)


In [40]:
print(len(word_index))

11972


In [41]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Convertir textos en secuencias numéricas
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

# Asegurarse de que las secuencias tengan la longitud máxima
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

# Confirmar que las etiquetas son enteros
y_train = y_train.astype(int)
y_val = y_val.astype(int)


In [42]:
print(f"Tamaño de las secuencias de entrenamiento: {X_train_pad.shape}")
print(f"Primer ejemplo de secuencia: {X_train_pad[0]}")

Tamaño de las secuencias de entrenamiento: (5087, 25)
Primer ejemplo de secuencia: [   2 1705 1638   11  379 4597  124    4   13   75   34    1  971    0
    0    0    0    0    0    0    0    0    0    0    0]


In [43]:
import tensorflow as tf
from tensorflow.keras import backend as K

# Función para calcular F1-Score
def f1_score(y_true, y_pred):
    # Convertir y_true y y_pred al mismo tipo
    y_true = K.cast(y_true, 'float32')
    y_pred = K.round(y_pred)  # Redondear a 0 o 1

    # Calcular verdaderos positivos, falsos positivos y falsos negativos
    tp = K.sum(y_true * y_pred, axis=0)  # Verdaderos positivos
    fp = K.sum((1 - y_true) * y_pred, axis=0)  # Falsos positivos
    fn = K.sum(y_true * (1 - y_pred), axis=0)  # Falsos negativos

    # Calcular precisión y recall
    precision = tp / (tp + fp + K.epsilon())  # Evitar división por cero
    recall = tp / (tp + fn + K.epsilon())

    # Calcular F1-Score
    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    return K.mean(f1)  # Promedio del F1-Score para todas las clases



In [44]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l1

model = Sequential([
    Embedding(
        input_dim=MAX_NUM_WORDS,       # Tamaño del vocabulario
        output_dim=EMBEDDING_DIM,     # Dimensión de los embeddings
        weights=[embedding_matrix],   # Matriz de embeddings preentrenados
        input_length=MAX_SEQUENCE_LENGTH,  # Longitud máxima de las secuencias
        trainable=True                # Hacer los embeddings entrenables
    ),
    #LSTM(128, return_sequences=True, dropout=0.2),
    #BatchNormalization(),
    LSTM(64, dropout=0.2),
    BatchNormalization(),
    Dense(32, activation='relu', kernel_regularizer=l1(0.01)),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Número de clases (ajusta según tu caso)
])

# Compilar el modelo
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=[f1_score]
)

model.build(input_shape=(None, MAX_SEQUENCE_LENGTH))
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 25, 300)             │       3,600,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 64)                  │          93,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,695,875 (14.10 MB)

 Trainable params: 3,695,747 (14.10 MB)

 Non-trainable params: 128 (512.00 B)

In [45]:
# Verificar si todas las capas tienen parámetros entrenables
for layer in model.layers:
    print(f"Layer: {layer.name}, Trainable: {layer.trainable}")

Layer: embedding_3, Trainable: True
Layer: lstm_4, Trainable: True
Layer: batch_normalization_4, Trainable: True
Layer: dense_6, Trainable: True
Layer: dropout_3, Trainable: True
Layer: dense_7, Trainable: True


In [46]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

reduce_lr = ReduceLROnPlateau(
    monitor='val_f1_score',
    factor=0.5,
    patience=2,
    min_lr=1e-5
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=50,  # Más épocas
    batch_size=32,
    class_weight=class_weights
)


Epoch 1/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - f1_score: 0.1588 - loss: 3.3454 - val_f1_score: 0.0000e+00 - val_loss: 2.3286
Epoch 2/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - f1_score: 0.1965 - loss: 1.9984 - val_f1_score: 0.0605 - val_loss: 1.4698
Epoch 3/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - f1_score: 0.3311 - loss: 1.1756 - val_f1_score: 0.3947 - val_loss: 1.0173
Epoch 4/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - f1_score: 0.4037 - loss: 0.7762 - val_f1_score: 0.4233 - val_loss: 0.9975
Epoch 5/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - f1_score: 0.4351 - loss: 0.5965 - val_f1_score: 0.4448 - val_loss: 0.9464
Epoch 6/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - f1_score: 0.4541 - loss: 0.4448 - val_f1_score: 0.4582 - val_loss: 1.0323
Epoch 7/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - f1_score: 0.4603 - loss: 0.3774 - val_f1_score: 0.4661 - val_loss: 1.1098
Epoch 8/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - f1_score: 0.4660 - loss: 0.3134 - val_f

In [47]:
# Evaluar en validación
results = model.evaluate(X_val_pad, y_val)
print(f"Loss: {results[0]}, F1-Score: {results[1]}")

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - f1_score: 0.4651 - loss: 2.5402
Loss: 2.6951518058776855, F1-Score: 0.4708959460258484


In [48]:
test_path = "/content/drive/My Drive/CompeticionPLN/FinancES_test_kaggle.csv"

test_df = pd.read_csv(test_path)
test_df.head()

,id,text
0,0,Las empresas chinas piden menos burocracia par...
1,1,Enrique Escalante (Havas): “Todos tenemos que ...
2,2,Banca March confía su gestión empresarial a la...
3,3,"Garamendi, sobre la ruptura de la fusión: ""Hay..."
4,4,El Gobierno vasco da por hecho la continuidad ...


In [49]:
# Convertir textos de prueba en secuencias numéricas
X_test_seq = tokenizer.texts_to_sequences(test_df["text"])

# Asegurarse de que las secuencias tengan la longitud máxima
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

# Realizar predicciones
predictions = model.predict(X_test_pad)

# Obtener las etiquetas predichas (la clase con mayor probabilidad)
predicted_labels = np.argmax(predictions, axis=1)

# Crear un DataFrame con las predicciones
submission_df = pd.DataFrame({'id': test_df['id'], 'label': predicted_labels})


51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step


In [50]:
submission_df.head()

,id,label
0,0,2
1,1,0
2,2,1
3,3,1
4,4,2


In [51]:
# Guardar las predicciones en un archivo CSV
submission_df.to_csv("submission.csv", index=False)